In [80]:
import os
from pathlib import Path
import shutil
import geopandas as gp
from shapely.geometry import mapping
import rioxarray as rxr
import xarray as xr
#import sys

In [83]:
IMG_DIR_IN = "/home/sandbox-cel/paraguay_lc/SegmentationData/time_series_vars"
POLY_DIR = "/home/downspout-cel/paraguay_lc/SegmentationData/user_train"
IMG_DIR_OUT = Path("/home/downspout-cel/paraguay_lc/SegmentationData/time_series_vars_crop")

In [84]:
##If passing from BASH:
#IMG_DIR_IN = sys.argv[1]
#POLY_DIR = sys.argv[2]
#IMG_DIR_OUT = sys.argv[3]

In [87]:
#IMG_DIR_OUT.mkdir(parents=True,exist_ok=True)

for gpkg in os.listdir(POLY_DIR):
    if 'grid' in gpkg:
        print('working on {}...'.format(gpkg))
        name = gpkg.split('_')
        yr = name[2][:4]
        cell = gpkg[:4]
        subdir_out = Path(os.path.join(IMG_DIR_OUT,name[0]))
        subdir_out.mkdir(parents=True,exist_ok=True)
        matchCellDir = os.path.join(IMG_DIR_IN,str(cell))
        subDirs = [f.name for f in os.scandir(matchCellDir) if f.is_dir()]
        for d in subDirs:
            subsubdir_out = Path(os.path.join(subdir_out,d))
            subsubdir_out.mkdir(parents=True,exist_ok=True)
            subsubdir_in = os.path.join(matchCellDir,d)
            for img in os.listdir(subsubdir_in):
                image = rxr.open_rasterio(os.path.join(subsubdir_in,img),
                                 masked=True).squeeze()
                crop_extent = gpd.read_file(os.path.join(POLY_DIR,gpkg))
                img_clipped = image.rio.clip(crop_extent.geometry.apply(mapping))
                img_clipped.rio.to_raster(os.path.join(subsubdir_out,img))
                ##To copy file without cropping, coment-out two lines above and use:
                #shutil.copyfile(os.path.join(subsubdir_in,img), os.path.join(subsubdir_out,img))

In [ ]:
###Add missing grid field to gpkg:

In [88]:
POLY_DIR = "/home/downspout-cel/paraguay_lc/SegmentationData/user_train"

import geopandas as gpd
import fiona

for item in os.listdir(POLY_DIR):
    if 'grid' in item: 
        gpkg = os.path.join(POLY_DIR,item)
        print(gpkg)
        src = gpd.read_file(gpkg)
        src['grid']=1
        print(src)
        src.to_file(os.path.join(POLY_DIR,'out',item), driver="GPKG")

In [76]:
##Copy new images back into original file. NOte overwrite -- may want to copy folder first
for img in os.listdir(os.path.join(POLY_DIR,'out')):
    shutil.move(os.path.join(POLY_DIR,'out',img), os.path.join(POLY_DIR,img))